In [138]:
import openml, fairlib
import fairlib as fl
from fairlib.preprocessing import LFR
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np
import torch
import torch.nn as nn

In [139]:
def evaluate_fairness(X_test, y_pred):
    """
    Evaluate the fairness metrics (SPD and DI) of the predictions.
    """
    X_test["income"] = y_pred
    dataset = fl.DataFrame(X_test)
    dataset.targets = "income"
    dataset.sensitive = "sex"

    spd = dataset.statistical_parity_difference()
    di = dataset.disparate_impact()
    return spd, di

In [140]:
def train_classifier(X, y):
    # Create a pipeline with scaling and logistic regression
    pipeline = make_pipeline(
        StandardScaler(),  # Scale features
        LogisticRegression(max_iter=1000, random_state=42)  # Increase max_iter
    )
    pipeline.fit(X, y)
    return pipeline

In [141]:
dataset = openml.datasets.get_dataset(179)
X, y, _, names = dataset.get_data(target=dataset.default_target_attribute)

INFO:openml.datasets.dataset:pickle write adult


In [142]:
imputer = SimpleImputer(strategy='most_frequent')
X_imputed = imputer.fit_transform(X)

In [143]:
X_discretized = X_imputed.copy()
for col in X.columns:
    if X[col].dtype == 'category':
        le = LabelEncoder()
        X_discretized[:, X.columns.get_loc(col)] = le.fit_transform(X_discretized[:, X.columns.get_loc(col)])

In [144]:
X = fairlib.DataFrame(X_discretized, columns=names)
X = X.drop(columns=["fnlwgt"])
y = y.apply(lambda x: x == ">50K").astype(int)

In [145]:
X_train, X_test, y_train, y_test, s_train, s_test = train_test_split(X, y, X["sex"], test_size=0.35, random_state=42)

In [146]:
X_train

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country
48077,3,3,10,16,4,9,1,4,1,0,0,2,38
42663,4,3,5,4,2,11,0,4,1,0,0,2,38
22140,2,5,9,13,2,12,0,4,1,0,0,2,38
32827,4,3,11,9,2,9,0,4,1,0,0,2,38
29875,0,3,1,7,4,7,3,4,0,0,0,0,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,1,3,9,13,2,3,0,1,1,0,2,3,38
44732,0,3,11,9,4,6,3,4,0,0,0,2,38
38158,0,3,11,9,0,0,1,4,0,0,0,2,38
860,0,3,1,7,4,0,3,4,0,0,0,0,38


In [147]:
num_features = X_train.shape[1]

In [148]:
lfr = LFR(
    input_dim=num_features,
    latent_dim=8,
    output_dim=num_features,
    alpha_z=1.0, # Weight for the fairness loss (Lz)
    alpha_x=1.0, # Weight for the reconstruction loss (Lx)
    alpha_y=1.0, # Weight for the classification loss (Ly)
)

In [149]:
s_train = s_train.astype(float)

In [150]:
lfr.fit(X_train, y_train, s_train, epochs=100)

X_train_transformed = lfr.transform(X_train.astype(float))
X_test_transformed = lfr.transform(X_test.astype(float))

Epoch [10/100], Loss: 1.7143, Fairness: 0.0012, Reconstruction: 1.0010, Classification: 0.7122
Epoch [20/100], Loss: 1.6762, Fairness: 0.0004, Reconstruction: 0.9955, Classification: 0.6803
Epoch [30/100], Loss: 1.6232, Fairness: 0.0005, Reconstruction: 0.9898, Classification: 0.6329
Epoch [40/100], Loss: 1.5486, Fairness: 0.0008, Reconstruction: 0.9743, Classification: 0.5735
Epoch [50/100], Loss: 1.4565, Fairness: 0.0006, Reconstruction: 0.9381, Classification: 0.5177
Epoch [60/100], Loss: 1.3495, Fairness: 0.0007, Reconstruction: 0.8669, Classification: 0.4819
Epoch [70/100], Loss: 1.2516, Fairness: 0.0010, Reconstruction: 0.7975, Classification: 0.4531
Epoch [80/100], Loss: 1.1706, Fairness: 0.0007, Reconstruction: 0.7381, Classification: 0.4319
Epoch [90/100], Loss: 1.0923, Fairness: 0.0005, Reconstruction: 0.6777, Classification: 0.4141
Epoch [100/100], Loss: 1.0242, Fairness: 0.0004, Reconstruction: 0.6240, Classification: 0.3998


/Users/valerio/Desktop/UNIVERSITÀ/TESI/master-thesis-dizio-ay2324/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/valerio/Desktop/UNIVERSITÀ/TESI/master-thesis-dizio-ay2324/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [151]:
# Train classifier on original data
clf_original = train_classifier(X_train, y_train)
y_pred_original = clf_original.predict(X_test)

# Train classifier on transformed data
clf_transformed = train_classifier(X_train_transformed, y_train)
y_pred_transformed = clf_transformed.predict(X_test_transformed)

# Evaluate fairness metrics
spd_original, di_original = evaluate_fairness(X_test.copy(), y_pred_original)
spd_transformed, di_transformed = evaluate_fairness(
    X_test.copy(), y_pred_transformed
)

In [154]:
print("Original Data: SPD:", spd_original, "DI:", di_original, "\n")
print("Transformed Data: SPD:", spd_transformed, "DI:", di_transformed)

Original Data: SPD: {(income=0, sex=0): 0.16823670667334933, (income=0, sex=1): -0.16823670667334933, (income=1, sex=0): -0.16823670667334928, (income=1, sex=1): 0.16823670667334928} DI: {(income=0, sex=0): 0.8249228997586134, (income=0, sex=1): 1.212234501300203, (income=1, sex=0): 5.305950303234102, (income=1, sex=1): 0.18846765288971448} 

Transformed Data: SPD: {(income=0, sex=0): 0.10430623198941369, (income=0, sex=1): -0.10430623198941369, (income=1, sex=0): -0.10430623198941363, (income=1, sex=1): 0.10430623198941363} DI: {(income=0, sex=0): 0.8875392770087573, (income=0, sex=1): 1.126710699914335, (income=1, sex=0): 2.438514587776331, (income=1, sex=1): 0.410085715711012}
